In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
data = pd.read_csv('train_data.csv')

In [3]:
data.columns

Index(['int_rate', 'installment', 'dti', 'pub_rec', 'revol_bal', 'revol_util',
       'open_acc', 'total_acc', 'mort_acc', 'collections_12_mths_ex_med',
       'annual_inc', 'loan_amnt', 'cr_line_period', 'issue_d_period',
       'emplength', 'target', 'loan_purpose'],
      dtype='object')

In [4]:
data_fin = data[['loan_amnt', 'dti', 'emplength', 'annual_inc', 'target']]

In [5]:
pd.options.display.float_format = '{:.5f}'.format
data_fin.describe()

,loan_amnt,dti,emplength,annual_inc,target
count,83472.00000,83472.00000,83472.00000,83472.00000,83472.00000
mean,18931722.25417,19.66488,5.83859,93983984.08654,1.00000
std,10857853.68223,10.31938,3.74536,103235601.55659,0.84783
min,1200000.00000,0.00000,0.00000,38400.00000,0.00000
25%,10800000.00000,12.83000,2.00000,57600000.00000,0.00000
50%,17400000.00000,19.09000,6.00000,78000000.00000,1.00000
75%,25200000.00000,25.94000,10.00000,111600000.00000,2.00000
max,48000000.00000,999.00000,10.00000,11487686400.00000,2.00000


In [6]:
y = data_fin[['target']]
x = data_fin.drop('target', axis = 1)

In [7]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2, random_state=42)

In [23]:
rf = RandomForestClassifier(random_state = 0, n_jobs = 1)

params = {
    'n_estimators' : [100, 150, 200, 250, 300, 400, 500],
    'max_depth' : [5, 6, 7, 8, 9],
    'min_samples_split' : [2, 3, 4, 5]
}

In [24]:
grid = GridSearchCV(rf, param_grid=params, cv = 2, n_jobs = 2, verbose = 2)
grid.fit(train_x, train_y)

Fitting 2 folds for each of 96 candidates, totalling 192 fits


c:\Users\jayiu\anaconda3\envs\screening\Lib\site-packages\numpy\ma\core.py:2892: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\jayiu\anaconda3\envs\screening\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(cv=2, estimator=RandomForestClassifier(n_jobs=1, random_state=0),
             n_jobs=2,
             param_grid={'max_depth': [3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4, 5],
                         'n_estimators': [100, 200, 300, 400, 500, 600]},
             verbose=2)

In [25]:
grid.best_estimator_

RandomForestClassifier(max_depth=6, min_samples_split=4, n_estimators=200,
                       n_jobs=1, random_state=0)

In [26]:
grid.best_score_

np.float64(0.4052403607185969)

In [11]:
grid.best_params_

{'max_depth': 7, 'min_samples_split': 2, 'n_estimators': 300}

In [12]:
estimator = grid.best_estimator_

In [13]:
pred = estimator.predict(test_x)

In [14]:
print('accuracy : {0:.4f}'.format(accuracy_score(test_y, pred)))
print('f1-score : {0:.4f}'.format(f1_score(test_y, pred, average='macro')))
print('precision : {0:.4f}'.format(precision_score(test_y, pred, average='macro')))
print('recall : {0:.4f}'.format(recall_score(test_y, pred, average='macro')))

accuracy : 0.4277
f1-score : 0.4275
precision : 0.4295
recall : 0.4278


In [15]:
import pickle

with open('tool_ml.pkl', 'wb') as f:
    pickle.dump(estimator, f)

In [16]:
model = rf.fit(train_x, train_y)

d:\anaconda_envs\screening\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [17]:
preds = model.predict(test_x)

In [18]:
print('accuracy : {0:.4f}'.format(accuracy_score(test_y, preds)))
print('f1-score : {0:.4f}'.format(f1_score(test_y, preds, average='macro')))
print('precision : {0:.4f}'.format(precision_score(test_y, preds, average='macro')))
print('recall : {0:.4f}'.format(recall_score(test_y, preds, average='macro')))

accuracy : 0.3885
f1-score : 0.3884
precision : 0.3884
recall : 0.3885


In [2]:
df = pd.read_csv('./tool_data.csv')

In [3]:
df_use = df[['loan_amnt', 'dti', 'emplength', 'annual_inc', 'target']]
Y = df_use[['target']]
X = df_use.drop('target', axis = 1)

In [4]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size = 0.2, random_state=42)

In [5]:
import xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier()

In [10]:
params = {
    'n_estimators' : [300, 350, 400, 450, 500, 600, 700],
    'max_depth' : [3, 4, 5, 6, 7, 8, 9, 10],
    'learning_rate' : [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1],
    'gamma' : [0, 1, 2, 3]
}

In [11]:
grid2 = GridSearchCV(xgb, param_grid=params, scoring="f1_macro", n_jobs=1, verbose = 2)
grid2.fit(train_X, train_Y)

Fitting 5 folds for each of 2240 candidates, totalling 11200 fits
[CV] END gamma=0, learning_rate=0.01, max_depth=3, n_estimators=300; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=3, n_estimators=300; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=3, n_estimators=300; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=3, n_estimators=300; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=3, n_estimators=300; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=3, n_estimators=350; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=3, n_estimators=350; total time=   0.2s
[CV] END gamma=0, learning_rate=0.01, max_depth=3, n_estimators=350; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=3, n_estimators=350; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=3, n_estimators=350; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=3, n_e

c:\Users\jayiu\anaconda3\envs\screening\Lib\site-packages\numpy\ma\core.py:2892: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=1,
             param_grid={'gamma': [0, 1, 2, 3],
                         'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06,
                                           0.07, 0.08, 0.09, 0.1],
                         'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [300, 350, 400, 450, 500, 600, 700]},
             scoring='f1_macro', verbose=2)

In [12]:
grid2.best_score_

np.float64(0.4332492938525993)

In [13]:
grid2.best_params_

{'gamma': 0, 'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 350}

In [14]:
esti = grid2.best_estimator_

In [15]:
pred = esti.predict(test_X)

In [16]:
print('accuracy : {0:.4f}'.format(accuracy_score(test_Y, pred)))
print('f1-score : {0:.4f}'.format(f1_score(test_Y, pred, average='macro')))
print('precision : {0:.4f}'.format(precision_score(test_Y, pred, average='macro')))
print('recall : {0:.4f}'.format(recall_score(test_Y, pred, average='macro')))

accuracy : 0.4450
f1-score : 0.4278
precision : 0.4517
recall : 0.4516
